In [1]:
import random
from random import sample
import re
import copy
from patterns import word_patterns,sent_patterns,cut_sentence
import pandas as pd


standard = pd.read_excel('standard.xlsx')
sent_info = pd.read_excel('sent_info.xlsx')

In [2]:
# 查看数据信息——同一text中有多少match
data2010 = pd.read_excel('data/2010_afterClean_mix.xlsx')
data2012 = pd.read_excel('data/2012_afterClean_mix.xlsx')
data2014 = pd.read_excel('data/2014_afterClean_mix.xlsx')
data2018 = pd.read_excel('data/2018_afterClean_mix.xlsx')

def textInfo(data):
    matchsPerText = {}
    pre_i = 0
    pre_text = data['text'][0]
    for i in range(1,len(data)):
        if (data['text'][i]!=pre_text):
            count = i-pre_i
            if count in matchsPerText.keys():
                matchsPerText[count] += 1
            else:
                matchsPerText[count] = 1
            pre_i = i
            pre_text = data['text'][i]
    matchsPerText[i+1-pre_i] += 1
    return matchsPerText
    
matchsPerText2010 = textInfo(data2010)
matchsPerText2012 = textInfo(data2012)
matchsPerText2014 = textInfo(data2014)
matchsPerText2018 = textInfo(data2018)
matchsPerText2010,matchsPerText2012,matchsPerText2014,matchsPerText2018

In [3]:
# 判断该句子是词级or句级
def isCharOrSent(label, match):
    '''
    return: 词级or句级， 对应的label序号(按照标准.xlsx，即从0开始)
    '''
    for i in range(len(standard)):
        if ((label==standard['grammar'][i]) and (match in standard['content'][i])):
            if (str(i+1) in word_patterns.keys()):
                return "Char", i
            elif (str(i+1) in sent_patterns.keys()):
                return "Sent", i
    return "Unknow", -1

isCharOrSent('词类副词程度副词','有点儿')

('Char', 60)

In [4]:
# 查找句级语法点在句子中对应位置
def matchIdx(text, match):
    ret_idx, ret_char = [], []
    idx = 0
    # 去除（）内容
    if (re.search('（.+）',match)!=None):
        tmp_match = match.replace('（','').replace('）','')
#         print(tmp_match)
        if (tmp_match in text):  # （）内的内容需要
            match = tmp_match
        else:   # （）内的内容不需要
            match = match.replace(re.search('（.+）',match).group(),'')
    
    match_content = re.split('[……|，]',match)
    text_cut = cut_sentence(text[:-1])
#     print(text_cut)
    while ('' in match_content):
        match_content.remove('')
    while (' ' in match_content):
        match_content.remove(' ')
#     print(match_content)
    for items in match_content:
        tmp_idx = 0
        # 对有'/'分割的查找其确切对应文字
        if ('/' in items):
            items = items.split('/')
#             print(items)
            flag = 0
            for item in items:
                if (flag==1):
                    break
                else:
                    if (item in text_cut):
                        idx = text.index(item)
                        ret_char.append(item)
                        flag = 1
        else:
#             print('debug:',items,text)
            idx = text.index(items)
            # 防止不同点对应到同一处
            if (len(ret_idx)>0):
                tmp_idx = ret_idx[-1]
                while (idx <= tmp_idx):
                    idx = text[tmp_idx+1:].index(items)+tmp_idx+1
            ret_char.append(items)
            
        ret_idx.append(idx)
    
    # 对text_cut分词没法正确识别的情况，直接查找位置
    if (ret_char==[]):
        for items in match_content:
            if ('/' in items):   # 对有'/'分割的查找其确切对应文字
                items = items.split('/')
                flag = 0
                for item in items:
                    if (flag==1):
                        break
                    else:
                        if (item in text):
                            ret_idx[-1] = text.index(item)
                            ret_char.append(item)
                            flag = 1
            else:
                ret_idx[-1] = text.index(items)
        
    return ret_idx,ret_char
        
matchIdx('因为一棵树不但是一台“吸水器”，还是一台“降雨机”','（是）……，还是……')

([16], ['还是'])

In [5]:
# 删除对应match ——词级：句子只有1个语法点—— 词间和词内都删除（如果允许）
#               ——句级：删除1处match的字符——词间和词内都删除
def deleteToken(text, label, match):
    kind, label_id = isCharOrSent(label, match)
    ret, ops = [], []  # 返回样本数组、对应操作标记数组
    ret_text, ret_text2 = "", ""
    
    if (kind=="Char"):    
        char_idx = text.index(match)
        ret_text = text[:char_idx]
        
        char_len = len(match)
        # 仅进行词间删除，即将该字删除
        if (char_len==1 and char_idx<len(text)-1):
            ret_text += text[char_idx+1:]
        elif (char_len>1):
            ret_text2 = ret_text
            # 词间删除
            if (char_idx+char_len < len(text)):
                ret_text += text[char_idx+char_len:]
            # 进行词内删除，删除词中某几个字
            delete_num = random.randint(1,char_len-1)  # 待删除的字数
            tmp = random.randint(0,char_len-delete_num)  # 从词内的第几个字开始删
            ret_text2 += text[char_idx:char_idx+tmp]
            if (char_idx+tmp+delete_num < len(text)):
                ret_text2 += text[char_idx+tmp+delete_num:]
            ret.append(ret_text2)
            ops.append([(char_idx+tmp,char_idx+tmp,ret_text2[char_idx+tmp]),label,'M',('null','null',match[tmp:tmp+delete_num])])       

        ret.append(ret_text)
        ops.append([(char_idx,char_idx,ret_text[char_idx]),label,'M',('null','null',match)])
        
    if (kind=="Sent"):
        match_idx, match_char = matchIdx(text, match)
        if (len(match_char)==1):
            ret_text = text[:match_idx[0]]
            char_len = len(match_char[0])   # 匹配字符的长度
            del_idx = 0
        # 句中有多处匹配，随机选一处删除
        else:
#             print(text,match,label,match_char)
            del_idx = random.randint(0,len(match_char)-1)  # 删除第几处的字符
            char_len = len(match_char[del_idx])
            ret_text = text[:match_idx[del_idx]]
            
        # 词内删除
        if (char_len>1):
            ret_text2 = ret_text
            delete_num = random.randint(1,char_len-1) 
            tmp = random.randint(0,char_len-delete_num) 
            ret_text2 += text[match_idx[del_idx]:match_idx[del_idx]+tmp]
            ret_text2 += text[match_idx[del_idx]+tmp+delete_num:]
            ret.append(ret_text2)
            idx1 = match_idx[del_idx]+tmp
            ops.append([(idx1,idx1,ret_text2[idx1]),label,'M',('null','null',match_char[del_idx][tmp:tmp+delete_num])])
           
        ret_text += text[match_idx[del_idx]+char_len:]
        ret.append(ret_text)
        idx2 = match_idx[del_idx]
        ops.append([(idx2,idx2,ret_text[idx2]),label,'M',('null','null',match_char[del_idx])])
                    
    return ret, ops
        
a,b = deleteToken('这样做，只能暂时解决问题，所以要想完全解决这个难题，还需要找更好的办法。','句子的类型复句因果复句','（由于）……，所以/因此……')
a,b

(['这样做，只能暂时解决问题，所要想完全解决这个难题，还需要找更好的办法。', '这样做，只能暂时解决问题，要想完全解决这个难题，还需要找更好的办法。'],
 [[(14, 14, '要'), '句子的类型复句因果复句', 'M', ('null', 'null', '以')],
  [(13, 13, '要'), '句子的类型复句因果复句', 'M', ('null', 'null', '所以')]])

In [6]:
# 删除操作中对op进行调整
def delOpAdjust(op,ret_text,del_idx,del_len):
    '''
    ret_text：删除操作后的text
    del_idx：待删除字在text中的索引位置
    del_len：待删除字的长度
    '''
    for i in range(len(op)):
        if (del_idx<op[i][0][0]):
            tp1 = (op[i][0][0]-del_len,op[i][0][1]-del_len,op[i][0][2])            
            op[i][0] = tp1
        # 对于M操作的op，若idx相同，需改其对应的下一个字符
        elif (del_idx==op[i][0][0] and op[i][2]=='M'):
            tp3 = (op[i][0][0],op[i][0][1],ret_text[del_idx])        
            op[i][0] = tp3
        if (op[i][3][0]!='null' and del_idx<op[i][3][0]):
            tp2 = (op[i][3][0]-del_len,op[i][3][1]-del_len,op[i][3][2])
            op[i][3] = tp2
    return op

delOpAdjust(b,'这样做，只能暂时解决问题，所想完全解决这个难题，还需要找更好的办法。',14,1)

[[(14, 14, '想'), '句子的类型复句因果复句', 'M', ('null', 'null', '以')],
 [(13, 13, '要'), '句子的类型复句因果复句', 'M', ('null', 'null', '所以')]]

In [7]:
# 删除对应match ——句子有多个语法点—— 以50%概率删除词间或词内（如果允许，否则直接词间删除）
def deleteToken2(text, label, match, op):
    '''
    op: 之前操作的所有修改记录
    '''
    kind, label_id = isCharOrSent(label, match)
    ret_text = ""  # 返回string
    isAllDel = random.randint(0,1)  # 0:词内删 1:词间删
    
    if (kind=="Char"):            
        char_idx = text.index(match)
        ret_text = text[:char_idx]
        
        char_len = len(match)        
        # 直接进行词间删除，即将该字删除
        if ((char_len==1 and char_idx<len(text)-1) or isAllDel):
            ret_text += text[char_idx+char_len:]
            op = delOpAdjust(op,ret_text,char_idx,len(match))
            op.append([(char_idx,char_idx,ret_text[char_idx]),label,'M',('null','null',match)])
        # 进行词内删除，删除词中某几个字
        elif (char_len>1 and isAllDel==0):
            delete_num = random.randint(1,char_len-1)  # 待删除的字数
            tmp = random.randint(0,char_len-delete_num)  # 从词内的第几个字开始删
            ret_text += text[char_idx:char_idx+tmp]
            if (char_idx+tmp+delete_num < len(text)):
                ret_text += text[char_idx+tmp+delete_num:]
            op = delOpAdjust(op,ret_text,char_idx+tmp,len(text[char_idx+tmp:char_idx+tmp+delete_num]))
            op.append([(char_idx+tmp,char_idx+tmp,ret_text[char_idx+tmp]),label,'M',('null','null',text[char_idx+tmp:char_idx+tmp+delete_num])])
        
    if (kind=="Sent"):
        match_idx, match_char = matchIdx(text, match)
        del_idx = random.randint(0,len(match_char)-1)  # 删除第几处的字符
        idx1 = match_idx[del_idx]
        
        ret_text = text[:idx1]
        if (len(match_char[del_idx])==1 or isAllDel):            
            ret_text += text[idx1+len(match_char[del_idx]):]
            op = delOpAdjust(op,ret_text,idx1,len(match_char[del_idx]))
            op.append([(idx1,idx1,ret_text[idx1]),label,'M',('null','null',match_char[del_idx])])
        elif (len(match_char[del_idx])>1 and isAllDel==0):
            delete_num = random.randint(1,len(match_char[del_idx])-1)  # 待删除的字数
            tmp = random.randint(0,len(match_char[del_idx])-delete_num)  # 从词内的第几个字开始删
            idx2 = idx1+tmp
            ret_text += text[idx1:idx2]
            if (match_idx[del_idx]+tmp+delete_num < len(text)):
                ret_text += text[idx2+delete_num:]
            op = delOpAdjust(op,ret_text,idx2,len(text[idx2:idx2+delete_num]))
            op.append([(idx2,idx2,ret_text[idx2]),label,'M',('null','null',text[idx2:idx2+delete_num])])
        
    return ret_text,op
        

a,b = deleteToken2('这样做，只能暂时解决问题，所以要想完全解决这个难题，还需要找更好的办法。','句子的类型复句因果复句','（由于）……，所以/因此……',[[(15, 15, '要'), '句子', 'M', ('null', 'null', '以')]])
a,b

('这样做，只能暂时解决问题，所要想完全解决这个难题，还需要找更好的办法。',
 [[(14, 14, '要'), '句子', 'M', ('null', 'null', '以')],
  [(14, 14, '要'), '句子的类型复句因果复句', 'M', ('null', 'null', '以')]])

In [8]:
def sentReplace(match, word, rep_num):
    '''
    word：待替换的词(list)
    rep_num：替换的个数
    '''
    ret = []
    
    if (rep_num==1):
        dataset_rep = sent_info[sent_info['slice_len']==1]
    else:
        dataset_rep = pd.concat([sent_info[sent_info['slice_len']==2],sent_info[sent_info['slice_len']==3]])
    dataset_rep.reset_index(inplace=True)
    
    j = random.randint(0,len(dataset_rep)-1)
    while(match==dataset_rep['match'][j]):
        j = random.randint(0,len(dataset_rep)-1)
        
    match_slice = dataset_rep['match_slice'][j]
    match_slice = match_slice.split('，')
    for i in range(rep_num):
        if ('/' in match_slice[i]):
            tmp_match = match_slice[i].split('/')
            tmp_j = random.randint(0,len(tmp_match)-1)
            ret.append(tmp_match[tmp_j])
        else:
            ret.append(match_slice[i])
            
    return ret

        
sentReplace('一……就……',['一','就'],2)

['在', '方面']

In [9]:
# 寻找同一语法类型对应的所有match
def getAllMatch(label):
    candidates = []
    for i in range(len(standard)):
        if (standard['grammar'][i]==label):
            candidates.extend(re.split('[、|\\\]',standard['content'][i]))  # 以、和\作为分隔符
    return candidates


# 替换
def repaceToken(text, label, match):
    kind, label_id = isCharOrSent(label, match)
    ret_text, ret_op = "", []
    
    if (kind=="Char"):
        char_idx = text.index(match)
        ret_text = text[:char_idx]
        
        # replace
        candidates = getAllMatch(label)
        if (match in candidates):
            candidates.remove(match)  # 剔除自身
        if (len(candidates)==0):  # 若无可替换的词语，则返回空string
            return "",[]
        rep_word = candidates[random.randint(0,len(candidates)-1)]  # 随机选取一个同类型词语替换
        ret_text += rep_word
        ret_op.append([(char_idx,char_idx+len(rep_word)-1,ret_text[char_idx:char_idx+len(rep_word)]),label,'S',('null','null',match)])
        
        char_len = len(match)
        if (char_idx+char_len < len(text)):
            ret_text += text[char_idx+char_len:]
            
    if (kind=="Sent"):
        match_idx, match_char = matchIdx(text, match)
        flag_rep = random.randint(0,1)  # 0:替换1处 1:替换2处
        # 替换一处
        if (len(match_char)==1 or flag_rep==0):
            if (len(match_char)==1):
                tmp_ret = sentReplace(match,match_char,1)
                idx1 = match_idx[0]
                ret_text = text[:idx1] + tmp_ret[0] + text[idx1+len(match_char[0]):]
                ret_op.append([(idx1,idx1+len(tmp_ret[0])-1,ret_text[idx1:idx1+len(tmp_ret[0])]),label,'S',('null','null',match_char[0])])
            else:
                rep_idx = random.randint(0,len(match_char)-1)  # 替换的位置
                tmp_ret = sentReplace(match,[match_char[rep_idx]],1)
                idx2 = match_idx[rep_idx]
                ret_text = text[:idx2] + tmp_ret[0] + text[idx2+len(match_char[rep_idx]):]
                ret_op.append([(idx2,idx2+len(tmp_ret[0])-1,ret_text[idx2:idx2+len(tmp_ret[0])]),label,'S',('null','null',match_char[rep_idx])])
        # 替换两处
        else:
            if (len(match_char)==2):
                tmp_ret = sentReplace(match,match_char,2)
                idx1, idx2 = match_idx[0], match_idx[1]
                idx3 = idx2+len(tmp_ret[0])-1
                ret_text = text[:idx1] + tmp_ret[0] + text[idx1+len(match_char[0]):idx2] + tmp_ret[1] + text[idx2+len(match_char[1]):]
                t1,t2 = ret_text[idx1:idx1+len(tmp_ret[0])],ret_text[idx3:idx3+len(tmp_ret[1])]
                if (t1 != match_char[0]):
                    ret_op.append([(idx1,idx1+len(tmp_ret[0])-1,t1),label,'S',('null','null',match_char[0])])
                if(t2 != match_char[1]):
                    ret_op.append([(idx3, idx3+len(tmp_ret[1])-1,t2),label,'S',('null','null',match_char[1])])  

            else:   # len(match_char)==3
                rep_idx = [random.randint(0,len(match_char)-1)]   # 替换的位置(list)
                tmp_idx2 = random.randint(0,len(match_char)-1)
                while (tmp_idx2==rep_idx[0]):
                    tmp_idx2 = random.randint(0,len(match_char)-1)
                rep_idx.append(tmp_idx2)
                rep_idx.sort()
                
                tmp_ret = sentReplace(match,[match_char[rep_idx[0]],match_char[rep_idx[1]]],2)
                idx1,idx2 = match_idx[rep_idx[0]],match_idx[rep_idx[1]]
                idx3 = idx2+len(tmp_ret[0])-1
                ret_text = text[:idx1] + tmp_ret[0] + text[idx1+len(match_char[rep_idx[0]]):idx2] + tmp_ret[1] + text[idx2+len(match_char[rep_idx[1]]):]
                t1,t2 = ret_text[idx1:idx1+len(tmp_ret[0])],ret_text[idx3:idx3+len(tmp_ret[1])]
                if (t1 != match_char[rep_idx[0]]):
                    ret_op.append([(idx1,idx1+len(tmp_ret[0])-1,t1),label,'S',('null','null',match_char[rep_idx[0]])])
                if (t2 != match_char[rep_idx[1]]):
                    ret_op.append([(idx3, idx3+len(tmp_ret[1])-1,t2),label,'S',('null','null',match_char[rep_idx[1]])])  
       
    return [ret_text],ret_op

a,b = repaceToken('连大人也做不到，孩子更做不到。','句子的类型复句递进复句','连……也/都……，……更……')
a,b

(['首先大人其次做不到，孩子更做不到。'],
 [[(0, 1, '首先'), '句子的类型复句递进复句', 'S', ('null', 'null', '连')],
  [(4, 5, '其次'), '句子的类型复句递进复句', 'S', ('null', 'null', '也')]])

In [10]:
# 替换操作中对op进行调整
def repOpAdjust(op,rep_idx,ori_len,rep_len,rep_char):
    '''
    rep_idx：待替换字在text中的索引位置
    ori_len：待替换字的长度
    rep_len：替换字的长度
    rep_char：替换的字
    '''                
    for i in range(len(op)):
        if (rep_idx==op[i][0][0]):
            tp3 = (op[i][0][0],op[i][0][1],rep_char[0])          
            op[i][0] = tp3   
        if (ori_len != rep_len):
            change_len = ori_len-rep_len
            if (rep_idx<op[i][0][0]):
                tp1 = (op[i][0][0]-change_len,op[i][0][1]-change_len,op[i][0][2])            
                op[i][0] = tp1                         
            if (op[i][3][0]!='null' and rep_idx<op[i][3][0]):
                tp2 = (op[i][3][0]-change_len,op[i][3][1]-change_len,op[i][3][2])
                op[i][3] = tp2
                
    return op

# repOpAdjust(b,14,1,2,'一二')

In [11]:
# 替换
def repaceToken2(text, label, match, op):
    kind, label_id = isCharOrSent(label, match)
    ret_text = ""
    
    if (kind=="Char"):
        char_idx = text.index(match)
        ret_text = text[:char_idx]
        
        # replace
        candidates = getAllMatch(label)
        if (match in candidates):
            candidates.remove(match)  # 剔除自身
        if (len(candidates)==0):  # 若无可替换的词语，则返回空string
            return "",[]
        rep_word = candidates[random.randint(0,len(candidates)-1)]  # 随机选取一个同类型词语替换
        ret_text += rep_word
        op = repOpAdjust(op,char_idx,len(match),len(rep_word),rep_word)
        op.append([(char_idx,char_idx+len(rep_word)-1,ret_text[char_idx:char_idx+len(rep_word)]),label,'S',('null','null',match)])
       
        char_len = len(match)
        if (char_idx+char_len < len(text)):
            ret_text += text[char_idx+char_len:]
            
    if (kind=="Sent"):
        match_idx, match_char = matchIdx(text, match)
        flag_rep = random.randint(0,1)  # 0:替换1处 1:替换2处
        flag_rep=1
        print(match_char)
        # 替换一处
        if (len(match_char)==1 or flag_rep==0):
            if (len(match_char)==1):
                tmp_ret = sentReplace(match,match_char,1)
                idx1 = match_idx[0]
                ret_text = text[:idx1] + tmp_ret[0] + text[idx1+len(match_char[0]):]
                op = repOpAdjust(op,idx1,len(match_char[0]),len(tmp_ret[0]),tmp_ret[0])
                op.append([(idx1,idx1+len(tmp_ret[0])-1,tmp_ret[0]),label,'S',('null','null',match_char[0])])
            else:
                rep_idx = random.randint(0,len(match_char)-1)  # 替换的位置
                tmp_ret = sentReplace(match,[match_char[rep_idx]],1)
                idx2 = match_idx[rep_idx]
                ret_text = text[:idx2] + tmp_ret[0] + text[idx2+len(match_char[rep_idx]):]
                op = repOpAdjust(op,idx2,len(match_char[rep_idx]),len(tmp_ret[0]),tmp_ret[0])
                op.append([(idx2,idx2+len(tmp_ret[0])-1,tmp_ret[0]),label,'S',('null','null',match_char[rep_idx])])    
        # 替换两处
        else:
            if (len(match_char)==2):
                tmp_ret = sentReplace(match,match_char,2)
                idx1, idx2 = match_idx[0], match_idx[1]
                idx3 = idx2+len(tmp_ret[0])-1
                ret_text = text[:idx1] + tmp_ret[0] + text[idx1+len(match_char[0]):idx2] + tmp_ret[1] + text[idx2+len(match_char[1]):]
                t1,t2 = ret_text[idx1:idx1+len(tmp_ret[0])],ret_text[idx3:idx3+len(tmp_ret[1])]
                if (t1 != match_char[0]):
                    op = repOpAdjust(op,idx1,len(match_char[0]),len(tmp_ret[0]),tmp_ret[0])
                    op.append([(idx1,idx1+len(tmp_ret[0])-1,t1),label,'S',('null','null',match_char[0])])
                if(t2 != match_char[1]):
                    op = repOpAdjust(op,idx3,len(match_char[1]),len(tmp_ret[1]),tmp_ret[1])
                    op.append([(idx3, idx3+len(tmp_ret[1])-1,t2),label,'S',('null','null',match_char[1])])  
            else:   # len(match_char)==3
                rep_idx = [random.randint(0,len(match_char)-1)]   # 替换的位置(list)
                tmp_idx2 = random.randint(0,len(match_char)-1)
                while (tmp_idx2==rep_idx[0]):
                    tmp_idx2 = random.randint(0,len(match_char)-1)
                rep_idx.append(tmp_idx2)
                rep_idx.sort()
                
                tmp_ret = sentReplace(match,[match_char[rep_idx[0]],match_char[rep_idx[1]]],2)
                idx1,idx2 = match_idx[rep_idx[0]],match_idx[rep_idx[1]]
                idx3 = idx2+len(tmp_ret[0])-1
                ret_text = text[:idx1] + tmp_ret[0] + text[idx1+len(match_char[rep_idx[0]]):idx2] + tmp_ret[1] + text[idx2+len(match_char[rep_idx[1]]):]
                t1,t2 = ret_text[idx1:idx1+len(tmp_ret[0])],ret_text[idx3:idx3+len(tmp_ret[1])]
                if (t1 != match_char[rep_idx[0]]):
                    op = repOpAdjust(op,idx1,len(match_char[rep_idx[0]]),len(tmp_ret[0]),tmp_ret[0])
                    op.append([(idx1,idx1+len(tmp_ret[0])-1,t1),label,'S',('null','null',match_char[rep_idx[0]])])
                if (t2 != match_char[rep_idx[1]]):
                    op = repOpAdjust(op,idx3,len(match_char[rep_idx[1]]),len(tmp_ret[1]),tmp_ret[1])
                    op.append([(idx3, idx3+len(tmp_ret[1])-1,t2),label,'S',('null','null',match_char[rep_idx[1]])])  
                
    return ret_text,op


a,b = repaceToken2('但随着年龄的增长，他逐渐成熟了，大学毕业后成了一名优秀的律师，真让人吃惊。','固定格式','（在）……以前/以后/前/后',[])
a,b

['后']


('但随着年龄的增长，他逐渐成熟了，大学毕业好成了一名优秀的律师，真让人吃惊。',
 [[(20, 20, '好'), '固定格式', 'S', ('null', 'null', '后')]])

In [12]:
# 当句级的句子切词有问题（即无法正确地将语法点处的字词分开）时，用该函数进行词间乱序操作
# 针对句子只有1处语法点
def bugDisorder(text, label, match_idx, match_char, ops):
    match_idx = match_idx[0]
    match_char = match_char[0]
    
    # 将原句text按语法点位置分为左右两部分
    left_text = text[:match_idx]  
    right_text = text[match_idx+len(match_char):]
    
    left_or_right = random.randint(0,1)  # 随机决定乱序后插入左侧or右侧句子
    left_cut = cut_sentence(left_text)
    right_cut = cut_sentence(right_text[:-1])
    ret_text = ""
    # 插入左侧
    if (len(right_cut)==0 or (left_or_right==0 and len(right_cut)>0)):
        disorder_idx = 0  # 定位待交换的idx
        ins_idx = random.randint(0,len(left_cut)-1)
        left_cut.insert(ins_idx,match_char)
        for i in range(len(left_cut)):
            if (ins_idx>i):
                disorder_idx += len(left_cut[i])
            ret_text += left_cut[i]
        ret_text += right_text
        ops.append([(disorder_idx,disorder_idx+len(match_char)-1,match_char),label,'W',(match_idx+len(match_char),match_idx+len(match_char),'None')])
    # 插入右侧
    else:
        disorder_idx = match_idx
        ins_idx = random.randint(0,len(right_cut)-1)
        right_cut.insert(ins_idx,match_char)
        ret_text += left_text
        for i in range(len(right_cut)):
            if (ins_idx>i):
                disorder_idx += len(right_cut[i])
            ret_text += right_cut[i]
        ops.append([(match_idx,match_idx,'None'),label,'W',(disorder_idx,disorder_idx+len(match_char)-1,match_char)])
    return ret_text+text[-1],ops

x,c=matchIdx('减肥不只是为了瘦，更是为了健康。', '……，更/还……')
a,b=bugDisorder('减肥不只是为了瘦，更是为了健康。','句子成分复句递进复句',x,c,[])
a,b

('减肥不只是为了瘦更，是为了健康。。', [[(8, 8, '更'), '句子成分复句递进复句', 'W', (10, 10, 'None')]])

In [13]:
# 打乱 ——句子只有1个语法点—— 词内(允许的话，随意一种)+词间(所有可能)
def disorderToken(text, label, match):
    ret, ops = [], []
    text_cut = cut_sentence(text[:-1])
    kind, label_id = isCharOrSent(label, match)
    
    if (kind=="Char"):    
        idx = text.index(match)   # match在原句中的位置
        match_idx = text_cut.index(match)  # match在切词后数组中的位置
        text_cut.remove(match)
        
        disorder_idx = 0  # 定位乱序位置
        isPre = 1  # 定位插入点在原本位置前(1)or后(0)
        for i in range(len(text_cut)+1):
            if (i==match_idx):
                isPre = 0
                if(i<len(text_cut)):
                    disorder_idx += len(text_cut[i])
                continue
            else:
                tmp = copy.copy(text_cut)
                tmp.insert(i,match)
                if (isPre):
                    ops.append([(disorder_idx,disorder_idx+len(match)-1,match),label,'W',(idx+len(match),idx+len(match),'None')])
                    disorder_idx += len(text_cut[i])
                else:
                    ops.append([(idx,idx,'None'),label,'W',(disorder_idx,disorder_idx+len(match)-1,match)])
#                     ops.append([(disorder_idx,disorder_idx+len(match)-1,match),label,'W',(idx,idx,'None')])
                    if (i<len(text_cut)):
                        disorder_idx += len(text_cut[i])

                ret_text = ""
                for j in range(len(tmp)):
                    ret_text += tmp[j]
                ret.append(ret_text+text[-1])

        # 词内
        if (len(match)>1):
            change_idx = random.randint(1,len(match)-1)
            new_match = match[change_idx:]+match[:change_idx]
            text_cut.insert(match_idx,new_match)

            ret_text = ""
            for j in range(len(text_cut)):
                ret_text += text_cut[j]
            ret.append(ret_text+text[-1])  # 带上标点
            ops.append([(idx,idx+len(match[change_idx:])-1,match[change_idx:]),label,'W',(idx+len(match[change_idx:]),idx+len(match)-1,match[:change_idx])])
        
    if (kind=="Sent"):
        match_idx, match_char = matchIdx(text, match)
#         print(match_idx, match_char)
        tmp_idx = 0
        
        if (len(match_char)==1):
            # 只能词间交换
            if (len(match_char[0])==1):
                tmp_idx = 0
            # 词内交换
            else:
                change_idx = random.randint(1,len(match_char[0])-1)
                idx1 = match_idx[0]
                match1,match2 = match_char[0][change_idx:],match_char[0][:change_idx]
                new_match = match1+match2
                ret_text = text[:idx1]+new_match+text[idx1+len(match_char[0]):]
                ret.append(ret_text)  
                ops.append([(idx1,idx1+len(match1)-1,match1),label,'W',(idx1+len(match1),idx1+len(match_char[0])-1,match2)])
        else:
            tmp_idx = random.randint(0,len(match_char)-1)   # 词间交换的index(相对于match_idx数组来说)
            
            if (len(match_char)==2):  # 二者互换
                choose_idx = [0,1]
            else:   # 任选两个互换
                choose_idx = sample(range(0,len(match_char)),2)  # 随机选2个不重复数
                choose_idx.sort()
            idx1,idx2 = match_idx[choose_idx[0]],match_idx[choose_idx[1]]
            char1,char2 = match_char[choose_idx[0]],match_char[choose_idx[1]]
            if (char1!=char2):
                ret_text = text[:idx1]+char2+text[idx1+len(char1):idx2]+char1+text[idx2+len(char2):]
                ret.append(ret_text)  
                ops.append([(idx1,idx1+len(char2)-1,char2),label,'W',(idx2+len(char2)-len(char1),idx2+len(char2)-1,char1)])
            
            # 词内交换
            dandiIdx = []  # 记录词语长度>1的index(相对于match_idx数组来说)
            for i in range(len(match_char)):
                if (len(match_char[i])>1):
                    dandiIdx.append(i)
            if (len(dandiIdx)>0):
                choose_idx = dandiIdx[random.randint(0,len(dandiIdx)-1)] # 选一个idx交换
                change_idx = random.randint(1,len(match_char[choose_idx])-1)
                idx1 = match_idx[choose_idx]
                match1,match2 = match_char[choose_idx][change_idx:],match_char[choose_idx][:change_idx]
                new_match = match1+match2
                ret_text = text[:idx1]+new_match+text[idx1+len(match_char[choose_idx]):]
                ret.append(ret_text)  
                ops.append([(idx1,idx1+len(match1)-1,match1),label,'W',(idx1+len(match1),idx1+len(match_char[choose_idx])-1,match2)])
        
        # 词间交换
        if (match_char[tmp_idx] in text_cut):
            cut_idx = text_cut.index(match_char[tmp_idx])  # match在切词后数组text_cut中的位置
            text_cut.remove(match_char[tmp_idx])

            disorder_idx = 0  # 定位乱序位置
            isPre = 1  # 定位插入点在原本位置前(1)or后(0)
            for i in range(len(text_cut)+1):
                if (i==cut_idx):
                    isPre = 0
                    if(i<len(text_cut)):
                        disorder_idx += len(text_cut[i])
                    continue
                else:
                    tmp = copy.copy(text_cut)
                    match1 = match_char[tmp_idx]
                    idx1 = match_idx[tmp_idx]
                    tmp.insert(i,match1)
                    if (isPre):
                        ops.append([(disorder_idx,disorder_idx+len(match1)-1,match1),label,'W',(idx1+len(match1),idx1+len(match1),'None')])
                        disorder_idx += len(text_cut[i])
                    else:
                        ops.append([(idx1,idx1,'None'),label,'W',(disorder_idx,disorder_idx+len(match1)-1,match1)])
    #                     ops.append([(disorder_idx,disorder_idx+len(match1)-1,match1),label,'W',(idx1,idx1,'None')])
                        if (i<len(text_cut)):
                            disorder_idx += len(text_cut[i])

                    ret_text = ""
                    for j in range(len(tmp)):
                        ret_text += tmp[j]
                    ret.append(ret_text+text[-1])
        else:
            ret_text,ops = bugDisorder(text, label, match_idx, match_char, ops)
            ret.append(ret_text[:-1])
    return ret,ops
            

a,b = disorderToken('减肥不只是为了瘦，更是为了健康。','句子成分复句递进复句','……，更/还……')
a,b

(['减肥不只是为了瘦，是为了更健康'], [[(9, 9, 'None'), '句子成分复句递进复句', 'W', (12, 12, '更')]])

In [14]:
# 替换操作中对op进行调整
def disorderOpAdjust(op,dis_idx1,len1,dis_idx2,len2,char1,char2,ret_text):
    '''
    dis_idx1：第一个待替换位置
    len1：第一个待替换字的长度
    char1：第一个待替换字
    ret_text：修改后的句子
    '''                
    for i in range(len(op)):
        # ‘M’操作index重合情况
        if (len1==len2):
            if (dis_idx1==op[i][0][0]):
                if (char2=='None'):
                    tp1 = (dis_idx1,dis_idx1,ret_text[dis_idx1]) 
                else:
                    tp1 = (dis_idx1,dis_idx1,char2[0])
                op[i][0] = tp1    
            elif (dis_idx2==op[i][0][0]):
                if (char1=='None'):
                    tp1 = (dis_idx2,dis_idx2,ret_text[dis_idx2]) 
                else:
                    tp1 = (dis_idx2,dis_idx2,char1[0]) 
                op[i][0] = tp1
        else:
            change_len = len1-len2
            # 对‘W’需考虑op[i][0]和op[i][3]
            if (op[i][2]=='W'):
#                 print(op[i][0][0],dis_idx1,op[i][3][0],dis_idx2)
                if (dis_idx1<op[i][0][0]<dis_idx2<op[i][3][0]):
                    tp2 = (op[i][0][0]-change_len,op[i][0][1]-change_len,op[i][0][2]) 
                    op[i][0] = tp2
                elif (dis_idx1<op[i][0][0]<op[i][3][0]<dis_idx2):
                    tp2 = (op[i][0][0]-change_len,op[i][0][1]-change_len,op[i][0][2]) 
                    tp3 = (op[i][3][0]-change_len,op[i][3][1]-change_len,op[i][3][2]) 
                    op[i][0] = tp2
                    op[i][3] = tp3
                elif (op[i][0][0]<dis_idx1<op[i][3][0]<dis_idx2):
                    tp2 = (op[i][3][0]-change_len,op[i][3][1]-change_len,op[i][3][2]) 
                    op[i][3] = tp2
                
            # 对‘M’和‘S’均只用考虑op[i][0]
            else:
                if (dis_idx1<op[i][0][0] and max(dis_idx2,dis_idx2+len1-1)>=op[i][0][0]):
                    tp4 = (op[i][0][0]-change_len,op[i][0][1]-change_len,op[i][0][2])    
                    op[i][0] = tp4
                
                if(dis_idx1==op[i][0][0]):
                    tp4 = (dis_idx1,dis_idx1,ret_text[dis_idx1])
                    op[i][0] = tp4
                elif (dis_idx2-change_len==op[i][0][0]):
                    print(op[i][0][0],len(ret_text))
                    tp4 = (op[i][0][0],op[i][0][0],ret_text[op[i][0][0]])
                    op[i][0] = tp4                    
                        
    return op

# disorderOpAdjust(b,1,2,5,1,'亿','而','ABCDEFGHIJKLMN')

In [15]:
# 辅助切词，针对句子被修改过，match无法单独被分出来的情况
def cutFunc(text, match):
    match_idx = text.index(match)
    if (match_idx==0):
        text2 = text[match_idx+1:]
        text_cut = cut_sentence(text2)
        text_cut.insert(0,match)
    elif (match_idx==len(text)-1):
        text1 = text[:match_idx]
        text_cut = cut_sentence(text1)
        text_cut.append(match)
    else:
        text1 = text[:match_idx]
        text2 = text[match_idx+1:]
        text_cut = cut_sentence(text1)
        text_cut.append(match)
        text_cut.extend(cut_sentence(text2))
    return text_cut


# 获取交换的index
def getIndex(text_cut,insert_idx,match_idx):
    '''
    text_cut: 切词后的数组
    insert_idx: 待插入text_cut的index
    match_idx: match在text_cut中的index
    return: [插入idx,原idx]
    '''
    ret_idx1, ret_idx2 = 0, 0
    left_idx, right_idx = min(insert_idx,match_idx), max(insert_idx,match_idx)
    for i in range(right_idx):
        if (i==left_idx):
            ret_idx2 = ret_idx1
            ret_idx2 += len(text_cut[i])
        elif (i<left_idx):
            ret_idx1 += len(text_cut[i])
        else:
            ret_idx2 += len(text_cut[i]) 
    # 词内交换，ret_idx2=0
    if (ret_idx2==0):
        ret_idx2 = ret_idx1
        
    if (insert_idx<=match_idx):
        return [ret_idx1, ret_idx2]
    else:
        return [ret_idx2, ret_idx1]

# cut_sentence('您看这个沙发怎么样'),getIndexRange(cut_sentence('您看这个沙发怎么样'),0,4)       

def getText(text_cut):
    ret_text = ''
    for i in range(len(text_cut)):
        ret_text += text_cut[i]
    return ret_text

# 打乱 ——句子有多个语法点—— 以50%概率词间或词内交换（如果允许，否则直接词间交换）
# notice: 在依次修改语法点的过程中，可能会因语句的修改，导致cut_sentence切词错误，故用cutFunc函数辅助进行
def disorderToken2(text, label, match, op):
    text_cut = cut_sentence(text[:-1])
    ret_text = ""
    kind, label_id = isCharOrSent(label, match)
    
    if (kind=="Char"):        
        if (match not in text_cut):
            text_cut = cutFunc(text[:-1], match)
            
        match_idx = text_cut.index(match)  # match在text_cut中的位置
        text_cut.remove(match)

        isFlag = random.randint(0,1)  # 0:词内交换 1:词间交换
        # 词间交换
        if (len(match)==1 or isFlag):
            insert_idx = random.randint(0,len(text_cut))
            while (insert_idx==match_idx):
                insert_idx = random.randint(0,len(text_cut))
            idxRande = getIndex(text_cut,insert_idx,match_idx)
            text_cut.insert(insert_idx,match)
            ret_text = getText(text_cut)
            if (idxRande[0]<idxRande[1]):
                op = disorderOpAdjust(op,idxRande[0],0,idxRande[1],len(match),'',match,ret_text)
                op.append([(idxRande[0],idxRande[0]+len(match)-1,match),label,'W',(idxRande[1]+len(match),idxRande[1]+len(match),'None')])
            else:
                op = disorderOpAdjust(op,idxRande[1],len(match),idxRande[0],0,match,'',ret_text)
                op.append([(idxRande[1],idxRande[1],'None'),label,'W',(idxRande[0],idxRande[0]+len(match)-1,match)])
                            
        # 词内交换
        elif (isFlag==0):
            change_idx = random.randint(1,len(match)-1)
            new_match = match[change_idx:]+match[:change_idx]
            text_cut.insert(match_idx,new_match)
            ret_text = getText(text_cut)
            idxRande = getIndex(text_cut,match_idx,match_idx)
            op = disorderOpAdjust(op,idxRande[0],len(match[:change_idx]),idxRande[1],len(match[change_idx:]),match[:change_idx],match[change_idx:],ret_text)
            op.append([(idxRande[0],idxRande[0]+len(match[change_idx:])-1,match[change_idx:]),label,'W',(idxRande[0]+len(match[change_idx:]),idxRande[0]+len(match)-1,match[:change_idx])])

        
    if (kind=="Sent"):
        match_idx, match_char = matchIdx(text, match)
        isFlag = random.randint(0,2)  # 0:词内交换 1:词间交换 2:词语互换
        
        dandiIdx = []  # 记录词语长度>1的index(相对于match_idx数组来说)
        for i in range(len(match_char)):
            if (len(match_char[i])>1):
                dandiIdx.append(i)
                                                
        # 词内交换
        if (isFlag==0 and len(dandiIdx)>0):
            choose_idx = dandiIdx[random.randint(0,len(dandiIdx)-1)]
            change_idx = random.randint(1,len(match_char[choose_idx])-1)            
            idx1 = match_idx[choose_idx]
            char1, char2 = match_char[choose_idx][change_idx:], match_char[choose_idx][:change_idx]
            new_match = char1+char2
            ret_text = text[:idx1]+new_match+text[idx1+len(match_char[choose_idx]):-1]
            op = disorderOpAdjust(op,idx1,len(char2),idx1+change_idx,len(char1),char2,char1,ret_text)
            op.append([(idx1,idx1+len(char1)-1,char1),label,'W',(idx1+len(char1),idx1+len(new_match)-1,char2)])
            
        # 词语互换
        elif (len(match_char)>1 and isFlag==2):          
            if (len(match_char)==2):  # 二者互换
                choose_idx = [0,1]
            else:   # 任选两个互换
                choose_idx = sample(range(0,len(match_char)),2)  # 随机选2个不重复数
                choose_idx.sort()
            idx1,idx2 = choose_idx[0],choose_idx[1]
            idx3,idx4 = match_idx[idx1],match_idx[idx2]
            char1,char2 = match_char[idx1],match_char[idx2]
            ret_text = text[:idx3]+char2+text[idx3+len(char1):idx4]+char1+text[idx4+len(char2):-1]
            op = disorderOpAdjust(op,idx3,len(char1),idx4,len(char2),char1,char2,ret_text)
            op.append([(idx3,idx3+len(char2)-1,char2),label,'W',(idx4-len(char1)+len(char2),idx4+len(char2)-1,char1)])
                        
        # 词间交换
        else:
            if (len(match_char)==1):
                tmp_idx = 0
            else:
                tmp_idx = random.randint(0,len(match_char)-1)   # 词间交换的index(相对于match_idx数组来说)
         
            if (match_char[tmp_idx] in text_cut):  # 对句子分词正确的情况
                cut_idx = text_cut.index(match_char[tmp_idx])  # match在切词后数组text_cut中的位置
                text_cut.remove(match_char[tmp_idx])

                disorder_idx = 0  # 定位乱序待插入位置(相对text来说)
                insert_idx = random.randint(0,len(text_cut))
                while (insert_idx==cut_idx):
                    insert_idx = random.randint(0,len(text_cut))  
                for i in range(0,insert_idx):
                    disorder_idx += len(text_cut[i])
                text_cut.insert(insert_idx,match_char[tmp_idx])
                ret_text = getText(text_cut)
                len1 = len(match_char[tmp_idx])
                if (insert_idx<cut_idx):
                    op = disorderOpAdjust(op,disorder_idx,0,match_idx[tmp_idx],len1,'',match_char[tmp_idx],ret_text)
                    op.append([(disorder_idx,disorder_idx+len1-1,match_char[tmp_idx]),label,'W',(match_idx[tmp_idx]+len1,match_idx[tmp_idx]+len1,'None')])
                else:
                    op = disorderOpAdjust(op,match_idx[tmp_idx],len1,disorder_idx,0,match_char[tmp_idx],'',ret_text)
                    op.append([(match_idx[tmp_idx],match_idx[tmp_idx],'None'),label,'W',(disorder_idx,disorder_idx+len1-1,match_char[tmp_idx])])
            else:   # 不能正确切词得到匹配项，则返回原句
                return text, op
    return ret_text+text[-1],op
            

a,b = disorderToken2('但随着年龄的增长，他逐渐成熟了，大学毕业后成了一名优秀的律师，真让人吃惊。','固定格式','（在）……以前/以后/前/后',[])
a,b

('但随着年龄的增长，他逐渐成熟了，大学毕业后成了一名优秀的律师，真让人吃惊。', [])

In [16]:
# 3种负样本构造方法
BUILD_MODE = {
    0: deleteToken2,  # 以50%概率进行词内删除
    1: repaceToken2,
    2: disorderToken2  # 以50%概率进行词内交换
}


# 负样本构造方法对应的option
OP_MODE = {
    0: 'M',  # delete--missing
    1: 'S',  # replace--selection
    2: 'W'   # disorder
}

In [17]:
# 随机方法组合：针对text中需要修改的多个match位置，以随机的方法决定每个位置应用哪种方法。若为-1，则不进行修改
def methodSet(match_num, change_num):
    '''
    match_num：text的match数
    change_num: text中需要修改的match数
    '''
    ret_num = random.randint(4,5)  # 对该text改写4 or 5次
    ret = []
    i = 0
    while (i<ret_num):
        ret_tmp = [random.randint(0,len(BUILD_MODE)-1) for j in range(change_num)]  # 随机决定每组具体操作
        for j in range(match_num-change_num):  # 插入不做修改的指示
            ret_tmp.insert(random.randint(0,len(ret_tmp)),-1)
        if ret_tmp in ret:
            i -= 1  # 组合重复
        else:
            ret.append(ret_tmp)
        i += 1
    return ret

methodSet(3,3)

[[1, 2, 0], [0, 1, 0], [1, 0, 0], [1, 1, 0]]

In [18]:
source, target, ops = [], [], []  # 错误句子，正确句子，操作类型，语法类型

In [19]:
# 负样本构建
data = pd.read_excel('data/2018_afterClean_mix.xlsx')
pre_i = 0  # text的起始位置
pre_text = data['text'][pre_i]  # 上一条句子

for i in range(pre_i, len(data)+1):
    if (i<len(data) and pre_text == data['text'][i]):
        continue
    else:
        print(i)
        match_count = i-pre_i
        
        # 句中只有1处语法点，则delete、replace、disorder全做
        if (match_count==1):
            delSent,op1 = deleteToken(pre_text,data['label'][pre_i],data['match'][pre_i])
            repSent,op2 = repaceToken(pre_text,data['label'][pre_i],data['match'][pre_i])
            disorderSent,op3 = disorderToken(pre_text,data['label'][pre_i],data['match'][pre_i])
            source.extend(delSent)
            ops.extend(op1)
            del_num,order_num = len(delSent), len(disorderSent)
            if (repSent!=""):
                source.extend(repSent)
                ops.append(op2)
                target.extend([pre_text for j in range(del_num+1+order_num)])
            else:
                target.extend([pre_text for j in range(del_num+order_num)])
            source.extend(disorderSent)
            ops.extend(op3)
        # 句中不止1处语法点，随机选4-5种组合操作
        else:
            isChar = 1
            # 每处语法点都做修改
            if (match_count<=2):
                methods = methodSet(match_count, match_count)
                for method in methods:
                    processed_text = pre_text
                    method_id = 0
                    tmp_op = []
                    op_num = 0
                    for tmp_method in method:
                        print("1:",tmp_method,processed_text,tmp_op,data['match'][pre_i+method_id])
                        processed_text,tmp_op = BUILD_MODE[tmp_method](processed_text,data['label'][pre_i+method_id],data['match'][pre_i+method_id],tmp_op)
                        op_num = len(tmp_op)
                        print(processed_text,tmp_op)
                        if (processed_text==""):
                            isChar = 0
                            break   
                        method_id += 1
                    if (isChar==0): 
                        break
                    else:
                        source.append(processed_text)
                        target.append(pre_text)
                        ops.append(tmp_op)

            # 随机选[2,5] or [2,match_count]个语法点做修改
            else:
                # [2,5] 
                if (match_count>=5):
                    change_count = random.randint(2,5)  # 修改数
                # [2,match_count] 
                else:
                    change_count = random.randint(2,match_count)

                methods = methodSet(match_count, change_count)
                for method in methods:
                    processed_text = pre_text
                    method_id = 0
                    tmp_op = []
                    op_num = 0
                    for tmp_method in method:
                        print("2:",tmp_method,processed_text,data['match'][pre_i+method_id],tmp_op)
                        if (tmp_method==-1):
                            method_id += 1
                            continue
                        else:
                            processed_text,tmp_op = BUILD_MODE[tmp_method](processed_text,data['label'][pre_i+method_id],data['match'][pre_i+method_id],tmp_op)
                            op_num = len(tmp_op)
                            if (processed_text==""):
                                isChar = 0
                                break   
                            method_id += 1
                    if (isChar==0): 
                        break
                    else:
                        source.append(processed_text)
                        target.append(pre_text)
                        new_method = list(filter(lambda x : x != -1, method))  # 去除所有-1
                        ops.append(tmp_op)                               

        # 更新    
        if (i<len(data)):
            pre_i = i
            pre_text = data['text'][i]

In [20]:
# len(source),len(target),len(ops)
df = pd.DataFrame({'source':source, 'target':target, 'op':ops})
df.to_excel('data/result_2018_mix.xlsx')